In [1]:
import os

In [2]:
%pwd

'd:\\recommendation-engine\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\recommendation-engine'

In [5]:
import pandas as pd

In [6]:
data1 = pd.read_csv("artifacts/data_ingestion/generated_data/bookings.csv")
data2 = pd.read_csv("artifacts/data_ingestion/generated_data/comments.csv")
data3 = pd.read_csv("artifacts/data_ingestion/generated_data/likes.csv")
data1.head()
data2.head()
data3.head()

,user_id,liked_event_id,liked_event_organizer_id,like_id
0,beca0aa6-bebd-4c34-9c5b-03edf7b71dc5,f9a6a40e-3b6b-4954-b45a-1c077358587b,60c5d29a-cd56-4a6d-83d8-8417f7dd8c45,3987e583-1fe8-4ec2-a973-914b2b497848
1,2b655047-c558-4fd6-a952-8e61969550e3,b81ab92e-0e6e-4b9b-ad50-6ac361b85a3f,dd0b548a-76c5-4bd0-8527-87ee47c7ae60,1b4c0bfb-f886-4e54-b24d-9b3dcea0e36f
2,a4a20a50-abfd-4ee8-9741-4e22eaf821d0,f604439a-66bc-4ad5-a35c-c4e5368dd7b0,e163a007-2139-427d-b878-58eaa2a5c81b,54e3b8d8-0653-4466-a05a-712a0069dcec
3,8cea2d02-9bed-4519-b33a-ba712767cf45,03cbfc57-67fa-4f76-aad8-d64fb28002f6,e87902c3-c62e-48b6-9b1a-851f9dc164ae,2b8f7076-743d-4208-a84e-9d214d36705d
4,6b91fcbb-b8bf-4ca7-90f4-1de83c0f5257,b541a36d-2e0b-4eda-ac96-eab938b5249c,81263b9c-871f-45a1-be63-93425afc20d3,d9e6f46c-45b3-4c15-acc4-767f9e382abd


In [7]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   user_id                    150000 non-null  object
 1   booked_event_id            150000 non-null  object
 2   booked_event_organizer_id  150000 non-null  object
 3   booking_id                 150000 non-null  object
dtypes: object(4)
memory usage: 4.6+ MB


In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    merged_data_url: Path
    unzip_data_dir: Path
    all_schema: dict

In [9]:
from src.hybrid_recommender.constants import *
from src.hybrid_recommender.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            merged_data_url = config.merged_data_url,
            all_schema=schema,
        )

        return data_validation_config

In [11]:
import os
from src.hybrid_recommender import logger


In [12]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.merged_data_url)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [13]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-06-30 21:47:14,730: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-30 21:47:14,744: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-30 21:47:14,751: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-30 21:47:14,753: INFO: common: created directory at: artifacts]
[2025-06-30 21:47:14,753: INFO: common: created directory at: artifacts/data_validation]
